<center><h1>Web Scraping Kijiji - Streamlined Version</h1><h3>Using Python and Beautiful Soup</h3></center>

In [1]:
import pandas as pd
from IPython.display import HTML
from bs4 import BeautifulSoup
import urllib.request as request
from ipywidgets import interact
pd.set_option("display.max_rows",1000)
pd.set_option("display.max_columns",20)
pd.set_option("display.max_colwidth", 200)

base_url = 'http://www.kijiji.ca'
toronto_url = 'http://www.kijiji.ca/h-city-of-toronto/1700273'
html_kijiji = request.urlopen(toronto_url)

soup_kijiji = BeautifulSoup(html_kijiji, 'lxml')

div_categories = soup_kijiji.find_all('a', class_='category-selected')

categories = {}
for item in div_categories:
    categories[item.get_text()] = base_url + item['href']
    
category_list = [key for key in categories.keys()]

pages = {'Page 1':'', 'Page 2':'page-2', 'Page 3':'page-3', 'Page 4':'page-4', 'Page 5':'page-5',
         'Page 6':'page-6', 'Page 7':'page-7', 'Page 8':'page-8', 'Page 9':'page-9'}

@interact
def kijiji_listings(category = sorted(category_list), page = sorted(pages)):
    if page == 'Page 1':
        print(categories[category])
        html_cars = request.urlopen(categories[category])
    else:
        url = categories[category]
        last_forward_slash = url.rfind('/')
        beginning_url = url[:last_forward_slash+1]
        ending_url = url[forward_slash:]
        print(beginning_url + pages[page] + ending_url)
        html_cars = request.urlopen(beginning_url + pages[page] + ending_url)

    soup_cars = BeautifulSoup(html_cars, 'lxml')
    
    #tables = soup_cars.find_all('table',  class_ = re.compile('regular-ad|top-'))
    tables = soup_cars.find_all('table')
    
    img_urls = []
    for table in tables[1:]:
        for row in table.find_all('td', class_='image'):
            try:
                img_urls.append("<img src='" + row.div.img['src'] + "'>")
            except:
                img_urls.append("<img src='" + row.img['src'] + "'>")
                
    titles = []
    for table in tables[1:]:
        for row in table.find_all('td', class_='description'):
            titles.append(row.a.get_text().strip())
            
    comments = []
    for table in tables[1:]:
        for row in table.find_all('td', class_='description'):
            comments.append(row.p.get_text().strip())
            
    details = []
    for table in tables[1:]:
        for row in table.find_all('td', class_='description'):
            for item in row.find_all('p', class_='details'):
                details.append(item.get_text().strip())
    
    prices = []
    for table in tables[1:]:
        for row in table.find_all('td', class_='price'):
            try:
                prices.append(float(row.get_text().replace('$','').replace(',','').strip()))
            except:
                prices.append(0.0)
    
    try:
        df = pd.DataFrame({'Price':prices, 'Image':img_urls, 'Title':titles, 'Comment':comments, 'Details':details})
        # Arrange the columns in a certain order
        df = df[['Image','Title','Comment','Details','Price']]
    # Some category listings don't have a price and title, so this script would bomb unless we leave them out
    except:
        df = pd.DataFrame({'Image':img_urls, 'Comment':comments, 'Details':details})
        # Arrange the columns in a certain order
        df = df[['Image','Comment','Details']]

    
    return HTML(df.to_html(escape=False))  # if escape is set to True, the images won't be rendered

http://www.kijiji.ca/b-cars-trucks/city-of-toronto/convertible__coupe__hatchback__other+body+type__sedan__wagon/c174l1700273a138


,Image,Title,Comment,Details,Price
0,,2012 Honda Civic EX AUT0 A/C SUNROOF ONLY 94K,"....SAFETY, E-TEST & CARPROOF INCLUDED...LEASED IN 2011 AND SERVICED IN 2013 AND 2014 IN A HONDA STORE, SEDAN, AUTO, A/C, POWER SUNROOF, ALLOY WHEELS, KEYLESS ENTRY AND MUCH MORE......**0 DOWN! No…",94379km | Automatic,11250
1,,2012 Volkswagen Jetta 2.5L COMFORTLINE AUT0 A/C CRUISE 56K,"...SAFETY, E-TEST & CARPROOF INCLUDED... LEASED IN 2011 AND SERVICED IN 2013 AND 2014 IN VOLKSWAGEN STORE, SEDAN, AUTOMATIC, A/C, ALLOY WHEELS, CRUISE CONTROL, HEATED SEATS, KEYLESS ENTRY AND MUCH…",56111km | Automatic,11990
2,,2013 Jaguar XJ AWD Leather NAV Sunroof Xenons Keyless_Go 19Alloy,Canadian car in prime condition with clean car proof! Loaded with: AWD! Superior leather seats! Front and rear heated seats! Navigation! Sunroof! Xenons! Keyless_Go! 19 alloys! Meridian audio! Dual…,37364km | Automatic | CarProof,51992
3,,"2015 Toyota Corolla LE Sedan, Auto, 22 000km 13300$",VEHICLE OPTIONS: Air Conditioning Cruise control Power locks Power mirrors Power steering Power windows Rear window defroster CD player Premium audio Heated seats Airbag: driver Airbag: passenger…,22000km | Automatic,13300
4,,2011 Lincoln MKZ *LEATHER HEATED/AC SEATS*LOW KM*THX AUDIO*,"Donway Ford Sales Limited 2011 Lincoln MKZ, in Grey, with ONLY 21,500 KMs!! 1 Owner. -LEATHER HEATED AND AIR CONDITIONED MEMORY SEATS -HEATED MIRRORS -AUTO DIMMING MIRRORS -SOLAR GLASS ON…",21500km | Automatic,18900
5,,2007 Honda Civic Coupe (2 door),"Selling a 2007 Honda Civic Coupe. Car is in amazing condition, very clean. Power locks. Power windows. Power steering. Remote w/keyless entry. New brakes. New battery. Sunroof. Cruise control. Air…",187973km | Automatic,6500
6,,2003 Toyota Corolla Sport Sedan,"Toyota Corolla Sport Sedan 2003 with security start feature with low mileage. I installed recetnely brakes, 4 set of all season tires ready for winter season ans also change the oil. I am selling my…",161000km | Automatic,4850
7,,2004 Honda Civic,"Very clean. Low mileage. Lots of new parts including Tires, Brakes, Battery and go on... It will be certified and emission test if you are interested Manual transmission. Asking Only $3.000 or best…",160000km | Manual,3000
8,,2008 HONDa Accord EX-L V6 LEATHER / SUNROOF / COUPE,"JUST ARRIVED! VERY WELL KEPT! PRICED RIGHT FOR QUICK SALE!! EQUIPPED WITH LEATHER SEATS, HEATED SEATS, SUNROOF, 6 SPEED MANUAL TRANSMISSION AND MUCH MUCH MORE!! CAR-PROOF VERIFIED, ASK FOR IT!!!!…",162981km | Manual,8950
9,,2005 CHEVROLET OPTRA WAGON**AUTO**A/C**LOADS OF FEATURES,"AUTO LINK MOTORS PROUD OMVIC & UCDA CERTIFIED DEALERSHIP 1614 Military Trail, Toronto. (647)618-8256 {2005 CHEVROLET OPTRA WAGON} >Very Spacious Extended Wagon >Fuel Efficient 4 Cylinder >Many…",130000km | Automatic,3995


<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>